# Prepare to run in Colab

In [1]:
# ! rm -r /content/movie-recommender-system
# import os

# os.chdir('/content/')

rm: cannot remove '/content/movie-recommender-system': No such file or directory


In [1]:
!git clone https://BatyshchevKirill@github.com/BatyshchevKirill/movie-recommender-system.git

Cloning into 'movie-recommender-system'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 90 (delta 49), reused 53 (delta 20), pack-reused 0
Receiving objects: 100% (90/90), 19.51 KiB | 6.50 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [2]:
import sys
import os

BASE_PATH = "/content/movie-recommender-system/"
os.chdir(BASE_PATH)
sys.path.append(BASE_PATH)

In [ ]:
!pip install -r requirements.txt

# Load the data

prepare the data

In [4]:
! python download_dataset.py
! python preprocess.py

Downloaded the dataset to data/raw


Create dataloader

In [5]:
from preprocess import create_loader

x_train = create_loader(dirpath='data/interim/', alpha=0.01, batch_size=16)

Train autoencoder

In [6]:
from autoencoder import train_and_encode

train_and_encode(x_train,
                input_dim =28,
                hidden_dim=16,
                enc_dim=8,
                epochs=100,
                noise=0.0,
                l1_weight=0.0,
                l2_weight=0.0,
                cpt_folder="models/",
                data_folder="data/interim/"
                )

100%|██████████| 100/100 [00:18<00:00,  5.26it/s, Epoch 100. Average MSE loss + regularization: 0.023748235082474804]


Train the final solution

In [8]:
from train import train_kmeans

train_kmeans("encoded_ae_16_8_100_0_0_0_0000_0_0000.csv")

Saved results to data/interim/ratings.csv


Visualize final mapping

In [9]:
import pandas as pd

ratings = pd.read_csv("data/interim/ratings.csv")
ratings

,movie_id,0,1,2,3,4,5,6,7,8,9
0,1,4.023810,3.905660,3.972973,3.720930,3.961538,3.586207,3.500000,3.800000,3.906250,4.250000
1,2,3.307692,3.032258,3.117647,3.142857,3.800000,3.714286,3.500000,3.000000,3.285714,2.714286
2,3,2.750000,2.947368,3.214286,3.444444,2.666667,2.714286,2.000000,3.333333,3.285714,2.250000
3,4,3.642857,3.460000,3.607143,3.565217,3.272727,3.000000,3.750000,3.600000,3.588235,4.125000
4,5,3.000000,3.166667,3.687500,2.909091,3.600000,3.800000,2.000000,3.000000,3.142857,4.000000
...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,4.666667,2.333333,1.000000,4.333333,4.000000,3.000000,3.000000,3.500000,3.000000,3.000000
1678,1679,2.972222,2.689394,3.000000,4.250000,3.700000,2.666667,3.000000,3.600000,3.550000,3.250000
1679,1680,3.555556,3.211310,2.000000,3.088889,3.125000,2.250000,3.000000,2.916667,3.916667,3.000000
1680,1681,3.125000,2.658575,3.000000,3.577778,3.242424,3.000000,3.000000,4.277778,3.472727,2.357143


# Inference

## Root mean squared error
On train set

In [53]:
!python benchmark/evaluate.py rmse -u data/interim/clustered_ae_16_8_100_0_0_0_0000_0_0000.csv -t data/interim/train.csv

Root mean squared error of predictions is: 0.9219278662363498


On test set

In [38]:
!python benchmark/evaluate.py rmse -u data/interim/clustered_ae_16_8_100_0_0_0_0000_0_0000.csv

Root mean squared error of predictions is: 1.105870230887717


We can see that the test set results are noticably worse. This may be because due to poor handling of "similar" movies

## Precision, recall

On train set

In [54]:
!python benchmark/evaluate.py pr -t data/interim/train.csv -u data/interim/clustered_ae_16_8_100_0_0_0_0000_0_0000.csv

Metrics of your data:
Precision -  0.7202
Recall    -  0.7575


On test set

In [55]:
!python benchmark/evaluate.py pr -u data/interim/clustered_ae_16_8_100_0_0_0_0000_0_0000.csv

Metrics of your data:
Precision -  0.6749
Recall    -  0.6945


The results are better than random and somehow comparable with the results of the benchmark article (0.77 and 0.8), so we casn consider this a success